### 절대모멘텀
- 주식 데이터를 로드 
- 파생변수 'STD-YM' 생성 -> Date 컬럼의 년-월 추출하여 대입
- 년-월 별 마지막 날의 데이터를 month_last_dt 데이터프레임에 대입
- 전월의 수정주가 값을 가지는 파생변수를 생성
- 전년도의 수정주가 값을 가지는 파생변수 생성
- 전월, 전년도의 수정주가를 가지고 거래내역을 생성
- 수익율 계산

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../../csv/AMZN.csv', index_col='Date')

In [ ]:
df.head()

In [ ]:
# index를 시계열 변경 
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')


In [ ]:
df['STD-YM'] = ""

In [ ]:
# STD-YM 컬럼을 생성 -> index에 있는 년-월 데이터를 추출
for i in range(len(df.index)):
    # i? -> 위치 값
    df['STD-YM'][i] = df.index[i].strftime('%Y-%m')

df.head()

In [ ]:
df.drop('STD-YM', axis=1, inplace=True)

In [ ]:
for i in df.index:
    # i? index 값
    df.loc[i, 'STD-YM'] = i.strftime('%Y-%m')

df.head()

In [ ]:
df.info()

In [ ]:
df.drop('STD-YM', axis=1, inplace=True)

In [ ]:
df['STD-YM'] = df.index.map(
    lambda x : x.strftime('%Y-%m')
)

In [ ]:
df.head()

In [ ]:
df2 = pd.read_csv('../../csv/AAPL.csv')
df2.head()

In [ ]:
from datetime import datetime

In [ ]:
# '1980-12-12' -> 시계열데이터 변환 ( 시계열 1980-12-12 ) -> 문자열 변환 ( 1980-12 )
df2['Date'].apply(
    lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m')
)

In [ ]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [ ]:
df2.info()

In [ ]:
df2['Date'].dt.strftime('%Y-%m')

In [ ]:
df.head()

In [ ]:
# STD-YM 데이터에서 다음 행의 데이터와 오늘의 데이터가 다른경우 
flag = df['STD-YM'].shift(-1) != df['STD-YM']
df.loc[flag]

In [ ]:
df.columns

In [ ]:
test_df = pd.DataFrame()
for i in range(len(df)-1):
    if df.iloc[i, 6] != df.iloc[i+1, 6]:
        data = df.iloc[[i]]
        test_df = pd.concat([test_df, data], axis=0)

test_df

In [ ]:
# 반복문 -> STD-YM 데이터에서 중복된 데이터를 제거하고 고유의 데이터들만 생성
list(
    set(
        df['STD-YM']
    )
)

In [ ]:
ym_list = df['STD-YM'].unique()

In [ ]:
ym_list[0]

In [ ]:
df.loc[df['STD-YM'] == ym_list[0]].tail(1)

In [ ]:
month_last_df = pd.DataFrame()

for i in ym_list:
    # print(i)
    flag = df['STD-YM'] == i
    data = df.loc[flag].tail(1)
    month_last_df = pd.concat([month_last_df, data], axis=0)

month_last_df

In [ ]:
# month_last_df에 파생변수 2개 생성
# 전월의 수정주가 (BF_1M)
# 전년도의 수정주가 (BF_12M)
# 결측치는 모두 0으로 대체

month_last_df['BF_1M'] = month_last_df['Adj Close'].shift(1).fillna(0)

In [ ]:
month_last_df['BF_12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

In [ ]:
month_last_df.iloc[[0, 1, 12]]

## 거래 내역 생성
- 매수 조건
    - 전월의 수정주가 / 전년도의 수정주가 - 1  의 값이 0보다 크고 무한대가 아닌경우

In [ ]:
import numpy as np

In [ ]:
# df 데이터프레임에 trade 컬럼을 생성해서 ""대입
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산식
    momentum_index = month_last_df.loc[i, 'BF_1M'] / \
        month_last_df.loc[i, 'BF_12M'] - 1
    # print(momentum_index)
    flag = True if( (momentum_index > 0) & (momentum_index != np.inf)) else False

    if flag:
        signal = 'buy'
    print(f'날짜 : {i}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}')
    df.loc[i:, 'trade'] = signal

df['trade'].value_counts()

In [ ]:
# df에 rtn 컬럼을 생성해서 1 데이터를 대입 
df['rtn'] = 1
# df.index 를 기준으로 하여 반복문을 실행 
for i in df.index:
    # 매수의 조건식을 지정 (전날의 trade가 '', 오늘의 trade가 'buy')
    if (df.shift().loc[i, 'trade'] == '') & (df.loc[i, 'trade'] == 'buy'): 
        # buy 변수를 생성하여 해당하는 날짜의 수정주가를 대입
        buy = df.loc[i, 'Adj Close']
        # print를 이용하여 매수일과 매수가를 출력
        print(f'매수일 : {i}, 매수가 : {buy}')
    # 매도의 조건식을 지정 (전 날의 trade가 'buy, 오늘의 trade가 '')
    elif (df.shift().loc[i, 'trade'] == 'buy') & (df.loc[i, 'trade'] == ''):
        # sell 변수를 생성하여 해당하는 날짜의 수정주가를 대입 
        sell = df.loc[i, 'Adj Close']
        # rtn 변수를 생성하여 sell / buy 값을 대입 
        rtn = sell / buy
        # 해당하는 날짜의 rtn 컬럼에 rtn 데이터를 대입 
        df.loc[i, 'rtn'] = rtn
        # print를 이용하여 매도일과 매도가 수익율을 출력
        print(f'매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}')


In [ ]:
# 누적 수익율 계산
df['acc_rtn'] = df['rtn'].cumprod()
df['acc_rtn'][-1]

### 절대 모멘텀을 함수화

1. 첫번째 함수 (create_YM)
    - 매개변수 2개 (데이터프레임, 기준이 되는 컬럼)
    - 컬럼에 Date가 포함되어있다면 Date컬럼을 인덱스로 변환
    - index를 시계열 데이터로 변경 
    - 기준이 되는 컬럼을 제외한 나머지 컬럼을 모두 제거 
    - 'STD-YM' 파생변수를 생성하여 index에서 년-월 추출하여 대입 
    - 수정이 된 데이터프레임을 리턴

2. 두번째 함수 (create_month_last())
    - 매개변수 3개(데이터프레임, 모멘텀기간, 시작시간)
    - 새로운 빈 데이터프레임을 생성 
    - 입력받은 데이터프레임에서 년-월별 마지막 데이터들을 새로운 데이터프레임에 대입
    - 파생변수(BF1)을 생성하여 전월의 주가를 대입 
    - 파생변수(BF2)을 생성하여 모멘텀기간(6 -> 6개월 전) 전의 주가를 대입 
    - 결측치는 0으로 대체
    - 데이터프레임를 시작시간을 기준으로 마지막 데이터까지 필터링
    - 위 과정에서 생성된 데이터프레임을 리턴 

In [ ]:
# 첫번째 함수
def create_ym(_df, _col = 'Adj Close'):
    # Date 컬럼이 존재한다면 index로 변경
    if 'Date' in _df.columns:
        _df.set_index('Date', inplace=True)
    # index를 시계열 데이터로 변경
    _df.index = pd.to_datetime(_df.index)

    # 기준이 되는 컬럼을 제외하고 모두 제거 
    _df = _df[[_col]]

    _df['STD-YM'] = _df.index.map(
        lambda x : x.strftime('%Y-%m')
    )

    return _df

In [ ]:
df = pd.read_csv('../../csv/AMZN.csv')

In [ ]:
price_df = create_ym(df, 'Close')

In [85]:
# 두번째 함수
def create_month_last(_df, _momentum = 12, _start = '2010-01-01'):
    result = pd.DataFrame()
    ym_list = _df['STD-YM'].unique()
    for i in ym_list:
        flag = _df['STD-YM'] == i
        data = _df.loc[flag].tail(1)
        result = pd.concat([result, data], axis=0)
    # result = _df.loc[_df.shift(-1)['STD-YM'] != _df['STD-YM']]
    # 파생변수 생성 
    # 전월의 데이터
    # 기준이 되는 컬럼의 이름
    col = _df.columns[0]
    result['BF1'] = result.shift(1)[col].fillna(0)
    result['BF2'] = result.shift(_momentum)[col].fillna(0)
    # 시작 시간부터 마지막 데이터까지 필터링 
    result = result.loc[_start:]
    return result

In [88]:
momentum_df = create_month_last(price_df)

- 세번째 함수 
    - 매개변수 3개 (1번함수의 결과(_df1), 2번 함수의 결과(_df2), 모멘텀 스코어)
    - _df1에는 trade 컬럼을 생성 "" 대입
    - _df1에는 rtn 컬럼을 생성 1 대입
    - _df2를 이용하여 momentum_index를 구하고 df1에 거래 내역 대입
    - 거래 내역을 기준으로 하여 수익율 계산
    - rtn를 기준으로 누적 수익율 계산
    - 최종 누적 수익율을 print를 이용하여 출력
    - _df1를 리턴

In [91]:
def create_trade(_df1, _df2, _score = 1):
    _df1['trade'] = ""
    _df1['rtn'] = 1

    # momentum_index를 생성
    for i in _df2.index:
        signal = ""

        # 절대 모멘텀을 계산
        momentum_index = _df2.loc[i, 'BF1'] / _df2.loc[i, 'BF2'] - _score

        # momentum_index가 0보다 크고 무한대가 아닐때 구매 조건
        flag = (momentum_index > 0) & (momentum_index != np.inf)

        if flag :
            signal = 'buy'
        # _df1의 trade에 signal 대입 
        _df1.loc[i:, 'trade'] = signal
        print(f'''날짜 : {i}, 모멘텀 인덱스 : , {momentum_index}, 
              flag : {flag}, signal : {signal}''')
    return _df1

In [92]:
price_df2 = create_trade(price_df, momentum_df)

날짜 : 2010-01-29 00:00:00, 모멘텀 인덱스 : , 1.2869772866371982, 
              flag : True, signal : buy
날짜 : 2010-02-26 00:00:00, 모멘텀 인덱스 : , 0.9356382476363905, 
              flag : True, signal : buy
날짜 : 2010-03-31 00:00:00, 모멘텀 인덱스 : , 0.6122004190577226, 
              flag : True, signal : buy
날짜 : 2010-04-30 00:00:00, 모멘텀 인덱스 : , 0.6861650404681461, 
              flag : True, signal : buy
날짜 : 2010-05-28 00:00:00, 모멘텀 인덱스 : , 0.7579178037676062, 
              flag : True, signal : buy
날짜 : 2010-06-30 00:00:00, 모멘텀 인덱스 : , 0.4996413220348399, 
              flag : True, signal : buy
날짜 : 2010-07-30 00:00:00, 모멘텀 인덱스 : , 0.27402051599765587, 
              flag : True, signal : buy
날짜 : 2010-08-31 00:00:00, 모멘텀 인덱스 : , 0.45202606350471575, 
              flag : True, signal : buy
날짜 : 2010-09-30 00:00:00, 모멘텀 인덱스 : , 0.3370822693114581, 
              flag : True, signal : buy
날짜 : 2010-10-29 00:00:00, 모멘텀 인덱스 : , 0.3219426028439123, 
              flag : True, signal : buy
날짜 : 201

C:\Users\moons\AppData\Local\Temp\ipykernel_3572\472568089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df1['trade'] = ""
C:\Users\moons\AppData\Local\Temp\ipykernel_3572\472568089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df1['rtn'] = 1


In [95]:
price_df2.loc[price_df2.shift()['trade'] != price_df2['trade']]

,Close,STD-YM,trade,rtn,acc_rtn
Date,,,,,
1997-05-15,1.958333,1997-05,,1,1
2010-01-29,125.410004,2010-01,buy,1,1
2012-03-30,202.509995,2012-03,,1,1
2012-04-30,231.899994,2012-04,buy,1,1
2014-10-31,305.459991,2014-10,,1,1
2015-03-31,372.100006,2015-03,buy,1,1


In [97]:
price_df2['trade'] == "buy"

Date
1997-05-15    False
1997-05-16    False
1997-05-19    False
1997-05-20    False
1997-05-21    False
              ...  
2019-06-18     True
2019-06-19     True
2019-06-20     True
2019-06-21     True
2019-06-24     True
Name: trade, Length: 5563, dtype: bool

In [98]:
# 네번째 함수 
def create_rtn(_df):
    col = _df.columns[0]

    for i in _df.index:
        # 구매한 날 조건식 
        if (_df.shift().loc[i, 'trade'] == '') & (_df.loc[i, 'trade'] == 'buy'):
            buy = _df.loc[i, col]
            print(f'매수일 : {i}, 매수가 : {buy}')
        # 판매한 날 조건식
        elif (_df.shift().loc[i, 'trade'] == 'buy') & (_df.loc[i, 'trade'] == ''):
            sell = _df.loc[i, col]
            rtn = sell / buy
            _df.loc[i, 'rtn'] = rtn
            print(f'매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}')

    # 누적수익율 계산
    _df['acc_rtn'] = _df['rtn'].cumprod()

    print(_df['acc_rtn'].iloc[-1])
    return _df

In [99]:
create_rtn(price_df2)

매수일 : 2010-01-29 00:00:00, 매수가 : 125.410004


C:\Users\moons\AppData\Local\Temp\ipykernel_3572\1291965725.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.6147834187135501' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  _df.loc[i, 'rtn'] = rtn


매도일 : 2012-03-30 00:00:00, 매도가 : 202.509995, 수익율 : 1.6147834187135501
매수일 : 2012-04-30 00:00:00, 매수가 : 231.899994
매도일 : 2014-10-31 00:00:00, 매도가 : 305.459991, 수익율 : 1.31720568737919
매수일 : 2015-03-31 00:00:00, 매수가 : 372.100006
2.1270019030151


C:\Users\moons\AppData\Local\Temp\ipykernel_3572\1291965725.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['acc_rtn'] = _df['rtn'].cumprod()


,Close,STD-YM,trade,rtn,acc_rtn
Date,,,,,
1997-05-15,1.958333,1997-05,,1.0,1.000000
1997-05-16,1.729167,1997-05,,1.0,1.000000
1997-05-19,1.708333,1997-05,,1.0,1.000000
1997-05-20,1.635417,1997-05,,1.0,1.000000
1997-05-21,1.427083,1997-05,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,1901.369995,2019-06,buy,1.0,2.127002
2019-06-19,1908.790039,2019-06,buy,1.0,2.127002
2019-06-20,1918.189941,2019-06,buy,1.0,2.127002
